# Scraping with Pandas

In [1]:
import pandas as pd

We can use the `read_html` function in Pandas to automatically scrape any tabular data from a page.

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_capitals_in_the_United_States'

In [3]:
tables = pd.read_html(url)
tables

[                             City                                Building  \
                   Albany Congress                         Albany Congress   
 0                Albany, New York                              Stadt Huys   
 1              Stamp Act Congress                      Stamp Act Congress   
 2              New York, New York                               City Hall   
 3      First Continental Congress              First Continental Congress   
 4      Philadelphia, Pennsylvania                        Carpenters' Hall   
 5     Second Continental Congress             Second Continental Congress   
 6      Philadelphia, Pennsylvania                       Independence Hall   
 7             Baltimore, Maryland                        Henry Fite House   
 8      Philadelphia, Pennsylvania                       Independence Hall   
 9         Lancaster, Pennsylvania                             Court House   
 10             York, Pennsylvania  Court House (now Colonial Co

#### What we get in return is a list of dataframes for any tabular data that Pandas found.

In [6]:
type(tables)

list

#### We can slice off any of those dataframes that we want using normal indexing.

In [8]:
df = tables[0]
df.head()

,City,Building,Start date,End date,Duration,Ref
,Albany Congress,Albany Congress,Albany Congress,Albany Congress,Albany Congress,Albany Congress
0,"Albany, New York",Stadt Huys,"June 19, 1754","July 11, 1754",22 days,[8]
1,Stamp Act Congress,Stamp Act Congress,Stamp Act Congress,Stamp Act Congress,Stamp Act Congress,Stamp Act Congress
2,"New York, New York",City Hall,"October 7, 1765","October 25, 1765",23 days,[9]
3,First Continental Congress,First Continental Congress,First Continental Congress,First Continental Congress,First Continental Congress,First Continental Congress
4,"Philadelphia, Pennsylvania",Carpenters' Hall,"September 5, 1774","October 26, 1774",1 month and 21 days,[10]


#### Drop all single header rows
`columns.get_level_values(0)` grabs the column names from the first header row

In [10]:
df.columns = df.columns.get_level_values(0)  # name the columns based on the first header row
df = df.loc[df.Ref.str.startswith("[")]  # remove any row that does not have a ref value
df.head()

,City,Building,Start date,End date,Duration,Ref
0,"Albany, New York",Stadt Huys,"June 19, 1754","July 11, 1754",22 days,[8]
2,"New York, New York",City Hall,"October 7, 1765","October 25, 1765",23 days,[9]
4,"Philadelphia, Pennsylvania",Carpenters' Hall,"September 5, 1774","October 26, 1774",1 month and 21 days,[10]
6,"Philadelphia, Pennsylvania",Independence Hall,"May 10, 1775","December 12, 1776","1 year, 7 months and 2 days",[11]
7,"Baltimore, Maryland",Henry Fite House,"December 20, 1776","February 27, 1777",2 months and 7 days,[12]


#### Split column values into two separate columns

In [11]:
columnsplit = df['City'].str.split(", ", expand=True)
df = df.assign(City=columnsplit[0],State=columnsplit[1])
df.head()

,City,Building,Start date,End date,Duration,Ref,State
0,Albany,Stadt Huys,"June 19, 1754","July 11, 1754",22 days,[8],New York
2,New York,City Hall,"October 7, 1765","October 25, 1765",23 days,[9],New York
4,Philadelphia,Carpenters' Hall,"September 5, 1774","October 26, 1774",1 month and 21 days,[10],Pennsylvania
6,Philadelphia,Independence Hall,"May 10, 1775","December 12, 1776","1 year, 7 months and 2 days",[11],Pennsylvania
7,Baltimore,Henry Fite House,"December 20, 1776","February 27, 1777",2 months and 7 days,[12],Maryland


#### Drop a column

In [12]:
df = df.drop(['Ref'], axis=1)
df.head()

,City,Building,Start date,End date,Duration,State
0,Albany,Stadt Huys,"June 19, 1754","July 11, 1754",22 days,New York
2,New York,City Hall,"October 7, 1765","October 25, 1765",23 days,New York
4,Philadelphia,Carpenters' Hall,"September 5, 1774","October 26, 1774",1 month and 21 days,Pennsylvania
6,Philadelphia,Independence Hall,"May 10, 1775","December 12, 1776","1 year, 7 months and 2 days",Pennsylvania
7,Baltimore,Henry Fite House,"December 20, 1776","February 27, 1777",2 months and 7 days,Maryland


#### Reset an index

In [13]:
df = df.reset_index(drop=True)
df.head()

,City,Building,Start date,End date,Duration,State
0,Albany,Stadt Huys,"June 19, 1754","July 11, 1754",22 days,New York
1,New York,City Hall,"October 7, 1765","October 25, 1765",23 days,New York
2,Philadelphia,Carpenters' Hall,"September 5, 1774","October 26, 1774",1 month and 21 days,Pennsylvania
3,Philadelphia,Independence Hall,"May 10, 1775","December 12, 1776","1 year, 7 months and 2 days",Pennsylvania
4,Baltimore,Henry Fite House,"December 20, 1776","February 27, 1777",2 months and 7 days,Maryland


In [14]:
df.loc[df.State=="New York"]

,City,Building,Start date,End date,Duration,State
0,Albany,Stadt Huys,"June 19, 1754","July 11, 1754",22 days,New York
1,New York,City Hall,"October 7, 1765","October 25, 1765",23 days,New York
13,New York,City Hall,"January 11, 1785","October 6, 1788","3 years, 11 months and 5 days",New York
14,New York,Federal Hall,"March 4, 1789","December 5, 1790","1 year, 9 months and 1 day",New York


## DataFrames as HTML

#### Pandas also has a `to_html` method that we can use to generate HTML tables from DataFrames.

In [16]:
html_table = df.to_html()
html_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>City</th>\n      <th>Building</th>\n      <th>Start date</th>\n      <th>End date</th>\n      <th>Duration</th>\n      <th>State</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Albany</td>\n      <td>Stadt Huys</td>\n      <td>June 19, 1754</td>\n      <td>July 11, 1754</td>\n      <td>22\xa0days</td>\n      <td>New York</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>New York</td>\n      <td>City Hall</td>\n      <td>October 7, 1765</td>\n      <td>October 25, 1765</td>\n      <td>23\xa0days</td>\n      <td>New York</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Philadelphia</td>\n      <td>Carpenters\' Hall</td>\n      <td>September 5, 1774</td>\n      <td>October 26, 1774</td>\n      <td>1\xa0month and 21\xa0days</td>\n      <td>Pennsylvania</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Philadelphia</td>\n      <td>In

#### You may have to strip unwanted newlines to clean up the table.

In [17]:
html_table.replace('\n', '')

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>City</th>      <th>Building</th>      <th>Start date</th>      <th>End date</th>      <th>Duration</th>      <th>State</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>Albany</td>      <td>Stadt Huys</td>      <td>June 19, 1754</td>      <td>July 11, 1754</td>      <td>22\xa0days</td>      <td>New York</td>    </tr>    <tr>      <th>1</th>      <td>New York</td>      <td>City Hall</td>      <td>October 7, 1765</td>      <td>October 25, 1765</td>      <td>23\xa0days</td>      <td>New York</td>    </tr>    <tr>      <th>2</th>      <td>Philadelphia</td>      <td>Carpenters\' Hall</td>      <td>September 5, 1774</td>      <td>October 26, 1774</td>      <td>1\xa0month and 21\xa0days</td>      <td>Pennsylvania</td>    </tr>    <tr>      <th>3</th>      <td>Philadelphia</td>      <td>Independence Hall</td>      <td>May 10, 1775</td>      <td>December 12, 1776</td>      <

You can also save the table directly to a file.

In [18]:
df.to_html('table.html')

In [14]:
# OSX Users can run this to open the file in a browser, 
# or you can manually find the file and open it in the browser
!open table.html